In [1]:
import numpy as np
import subprocess as sp
from openeye.oechem import *

from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer

# Load molecules into a list of `OEMol`s

I'm going to start by loading the host and guest SYBYL-formatted `mol2` files into a list of `OEMol`s. I created these SYBYL-formatted files from files containing AM1-BCC charges and GAFF v1.7 Lennard-Jones and bonded parameters:

```
antechamber -i bcd.mol2 -fi mol2 -o bcd-sybyl.mol2 -fo mol2 -at sybyl
antechamber -i ben.mol2 -fi mol2 -o ben-sybyl.mol2 -fo mol2 -at sybyl -dr n 
# Disable `acdoctor` to handle carboxylate group
```

I think it should be possible to read in GAFF-formatted files directly, using a specific forcefield flavor, but I don't see the relvant flavor listed in their [documentation](https://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html#section-molreadwrite-flavoredinputandoutput). A search for "GAFF" comes up empty (except for one blog post). If I try to load the GAFF files without any special flavor, I get many things with atom type "Du," which I surmise to be a dummy atom type. I believe this can be remedied by running the files through `OETriposAtomNames`, but it is another thing that could go wrong, so for these reasons, I believe it is more straight-forward to simply start with standard atom names.

In [219]:
def load_mol2(filename, name=None, add_tripos=True):
    ifs = oemolistream()
    molecules = []
    if not ifs.open(filename):
        print(f'Unable to open {filename} for reading...')
    for mol in ifs.GetOEMols():
        if add_tripos:
            OETriposAtomNames(mol)
        if name:
            mol.SetTitle(name)
        # Add all the molecules in this file to a list, but only return the first one.
        molecules.append(OEMol(mol))
    return molecules[0]

Even though we have properly formatted files, with unique atom names in each residue (there are seven residues in this host), if we don't call `OETriposAtomNames`, then we get `Exception: Error: Reference molecule must have unique atom names in order to create a Topology.` when trying to define the `System`.

In [220]:
host = load_mol2('original/bcd-sybyl.mol2', name='MGO', add_tripos=True)

In [221]:
def check_unique_atom_names(molecule):
    atoms = molecule.GetMaxAtomIdx()
    atom_names = set()
    for atom in range(atoms):
        atom_names.add(molecule.GetAtom(OEHasAtomIdx(atom)).GetName())
    print(f'{atoms} atoms in structure, {len(atom_names)} unique atom names.')
    assert atoms == len(atom_names)

In [222]:
check_unique_atom_names(host)

147 atoms in structure, 147 unique atom names.


In [223]:
guest = load_mol2('original/ben-sybyl.mol2', name='BEN', add_tripos=False)

In [224]:
molecules = [host, guest]

Now, I'm going to add water and ions to the molecules list.

In [225]:
def add_waters_and_ions():
    smiles = ['[Na+]', '[Cl-]', 'O']
    molecules = []
    for molecule in smiles:
        molecules.append(process_smiles(molecule))
    return molecules

def process_smiles(string, name=None, add_hydrogens=True, add_tripos=True, charge=True):
    mol = OEMol()
    OESmilesToMol(mol, string)
    if add_hydrogens:
        OEAddExplicitHydrogens(mol)
    if add_tripos:
        OETriposAtomNames(mol)
    if charge:
        for atom in mol.GetAtoms():
            atom.SetPartialCharge(atom.GetFormalCharge())
    if name:
        mol.SetTitle(name)
    return mol

In [226]:
water_and_ions = add_waters_and_ions()

In [227]:
molecules += water_and_ions

At this point, the molecules have been converted to `OEMol`s and they have charges, but the coordinates are not set. We could use a docking procedure like [this](https://github.com/openforcefield/openforcefield/blob/master/examples/host_guest_simulation/smirnoff_host_guest.ipynb) to set the position of the host and guest, or we could take the coordinates from an existing fully solvated system. I'm going to attempt the latter approach, because in general, we are not going to want to use a docking calculation to provide the coordinates for intermediate attach-pull-relase windows, where the host and guest are not  closely bound to each other.

# Read in (existing) coordinates to set the positions of the `OEMol`s

In [197]:
def create_pdb_with_conect(solvated_pdb, amber_prmtop, output_pdb, path='./'):
    """
    Create a PDB file containing CONECT records.
    This is not very robust, please manually check the `cpptraj` output.
    `cpptraj` must be in your PATH.
    Parameters
    ----------
    solvated_pdb : str
        Existing solvated structure from e.g., Mobley's Benchmark Sets repository
    amber_prmtop : str
        AMBER (or other) parameters for the residues in the solvated PDB file
    output_pdb : str
        Output PDB file name
    path : str
        Directory for input and output files
    """
    cpptraj = \
    f'''
    parm {amber_prmtop}
    trajin {solvated_pdb}
    trajout {output_pdb} conect
    '''

    cpptraj_input = output_pdb + '.in'
    cpptraj_output = output_pdb + '.out'

    with open(path + cpptraj_input, 'w') as file:
        file.write(cpptraj)
    with open(path + cpptraj_output, 'w') as file:
        p = sp.Popen(['cpptraj', '-i', cpptraj_input], cwd=path,
                     stdout=file, stderr=file)
        output, error = p.communicate()
    if p.returncode == 0:
        print('PDB file written by cpptraj.')
    elif p.returncode == 1:
        print('Error returned by cpptraj.')
        print(f'Output: {output}')
        print(f'Error: {error}')
    else:
        print(f'Output: {output}')
        print(f'Error: {error}')


def prune_conect(input_pdb, output_pdb, path='./'):
    """
    Delete CONECT records that correspond only to water molecules.
    This is necessary to be standards-compliant.
    This is not very robust.
    Parameters
    ----------
    input_pdb : str
        Input PDB file name
    output_pdb : str
        Output PDB file name
    path : str
        Directory for input and output files
    """
    p = sp.Popen(['grep', '-m 1', 'WAT', input_pdb], cwd=path, stdout=sp.PIPE)
    for line in p.stdout:
        first_water_residue = int(float(line.decode("utf-8").split()[1]))
        print(f'First water residue = {first_water_residue}')

    p = sp.Popen(['egrep', '-n', f'CONECT [ ]* {first_water_residue}', input_pdb],
                 cwd=path, stdout=sp.PIPE)
    for line in p.stdout:
        line_to_delete_from = int(float(line.decode("utf-8").split(':')[0]))
        print(f'Found first water CONECT entry at line = {line_to_delete_from}')

    with open(path + output_pdb, 'w') as file:
        sp.Popen(
         ['awk', f'NR < {line_to_delete_from}', input_pdb], cwd=path, stdout=file)

        sp.Popen(['echo', 'END'], cwd=path, stdout=file)

In [198]:
create_pdb_with_conect('original/solvated.inpcrd', 'original/solvated.prmtop', 'generated/solvated.pdb')

PDB file written by cpptraj.


In [199]:
prune_conect('generated/solvated.pdb', 'generated/solvated_conect.pdb')

First water residue = 175
Found first water CONECT entry at line = 9192


In [200]:
fixer = PDBFixer('generated/solvated_conect.pdb')

# Setup the force field and create an OpenMM `System` with the topology and molecules

One suggestion to get rid of the error `AttributeError: 'NoneType' object has no attribute 'used'` when trying to save the `System` as a `prmtop` from @swails is to use `rigidWater=False`: https://github.com/ParmEd/ParmEd/issues/930. In this case, it does not alleviate the issue, but leaving it on here does not seem to have any deleterious effects at the moment.

In [233]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml', 'forcefield/tip3p.ffxml') 
system = ff.createSystem(fixer.topology, molecules, 
                         nonbondedMethod=PME, 
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4,
                         rigidWater=False)

At this point, we should be able to run a simulation with SMIRNOFF99Frosst in OpenMM. Let's do a quick check.

In [295]:
integrator = openmm.VerletIntegrator(2.0*unit.femtoseconds)
simulation = app.Simulation(fixer.topology, system, integrator)
simulation.context.setPositions(fixer.positions) 

state = simulation.context.getState(getEnergy = True)
energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
print("Energy before minimization (kcal/mol): %.2g" % energy)

simulation.minimizeEnergy()
state = simulation.context.getState(getEnergy=True, getPositions=True)
energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
print("Energy after minimization (kcal/mol): %.2g" % energy)
minimized_coordinates = state.getPositions()

Energy before minimization (kcal/mol): 2.4e+03
Energy after minimization (kcal/mol): -2.2e+03


I'd also like to run a little MD, to see if the simulation is stable.

In [2]:
import mdtraj
from mdtraj.reporters import NetCDFReporter

In [4]:
md = dict(
    coordinates=minimized_coordinates,
    integrator=integrator,
    prefix='md',
    reporter_frequency=1000,
    nonbonded_method=app.NoCutoff,
    nonbonded_cutoff=None,
    constraints=app.HBonds,
    temperature=300 * unit.kelvin,
    steps=10000)
md['output'] = 'generated/' + md['prefix'] + '.nc'
md['h5'] = 'generated/' + md['prefix'] + '.h5'
md['data'] = 'generated/' + md['prefix'] + '.csv'

NameError: name 'minimized_coordinates' is not defined

In [303]:
simulation.context.setVelocitiesToTemperature(md['temperature'] * unit.kelvin)
# reporter = NetCDFReporter(md['output'], md['reporter_frequency'], 
#                          cell=True)
# simulation.reporters.append(reporter)

if not os.path.exists(md['h5']):
    simulation.reporters.append(mdtraj.reporters.HDF5Reporter(md['h5'], 1000))

simulation.reporters.append(
    app.StateDataReporter(
        md['data'],
        md['reporter_frequency'],
        step=True,
        potentialEnergy=True,
        temperature=True,
        density=True))

print('Running OpenMM MD...')
simulation.step(md['steps'])
print('MD completed.')

Running OpenMM MD...
MD completed.


In [4]:
import nglview as nv
from IPython import display

trajectory = mdtraj.load('generated/md.h5', 'r')
view = nv.show_mdtraj(trajectory, gui=True)
view

A Jupyter Widget

A Jupyter Widget

# Export the topology, system, and coordinates to AMBER files

In [243]:
import parmed as pmd

In [244]:
structure = pmd.openmm.topsystem.load_topology(fixer.topology, system, fixer.positions)

In [237]:
structure.save('generated/solvated_smirnoff.prmtop')

AttributeError: 'NoneType' object has no attribute 'used'

We see that there is still an issue with bond types, so @swails followed up with the suggestion of assigning these bonds to a dummy bond type and then later constraining them during the simulation. This does work to allow us to save the files, at least.

In [245]:
bond_type = pmd.BondType(1.0, 1.0, list=structure.bond_types)
structure.bond_types.append(bond_type)
for bond in structure.bonds:
    if bond.type is None:
        bond.type = bond_type

In [249]:
try:
    structure.save('generated/solvated_smirnoff.prmtop')
except OSError:
    print('File exists...')
    pass

File exists...


In [251]:
try:
    structure.save('generated/solvated_smirnoff.inpcrd')
except OSError:
    print('File exists...')
    pass

File exists...


In [252]:
try:
    structure.save('generated/solvated_smirnoff.mol2')
except OSError:
    print('File exists...')
    pass

File exists...


A few observations...

- The generated `prmtop` file is huge (1.1G)
- I think this is because every atom is getting a different parameter (even in, e.g., waters)
- Inside `%FLAG NONBONDED_PARM_INDEX` there appears to be an overflow, possibly due to the above issue
- Running a simulation with the generated `inpcrd` and `prmtop` in `pmemd` results in a seg fault:

```
Program received signal SIGSEGV: Segmentation fault - invalid memory reference.

Backtrace for this error:
#0  0x7FA5409FB6F7
#1  0x7FA5409FBD3E
#2  0x7FA53FF0026F
#3  0x49CFDE in __cit_mod_MOD_setup_crd_idx_tbl
#4  0x49D108 in __cit_mod_MOD_setup_cit
#5  0x479A48 in __pme_direct_mod_MOD_pme_list
#6  0x4913F2 in __pme_force_mod_MOD_pme_force
#7  0x4BD998 in __runmin_mod_MOD_runmin_master
#8  0x4E315B in MAIN__ at pmemd.F90:?
```

- There are no charges on the waters (because they don't have a formal charge, set above)
- The generated `mol2` file has numeric atom types
- Chimera crashes when trying to read the `inpcrd` and `prmtop` with:

```
Error: unexpected EOF in readln
Error looking for %FLAG RESIDUE_LABEL
```

I can't directly upload the `prmtop` to GitHub without going through large file storage, but I can spit out the head and tail of each `FLAG` section:

In [291]:
def grep_prmtop(prmtop, path):
    p = sp.Popen(['grep', '-A', '10', '-B', '10', 'FLAG', prmtop], cwd=path, stdout=sp.PIPE)
    for line in p.stdout:
        print(line.decode("utf-8").strip(),)

In [292]:
grep_prmtop('solvated_smirnoff.prmtop', path='generated/')

%VERSION  VERSION_STAMP = V0001.000  DATE = 02/05/18  15:13:40
%FLAG TITLE
%FORMAT(20a4)

%FLAG POINTERS
%FORMAT(10I8)
6804    6804    4495      93    2381     137     445     360       0       0
9804    2231      93     137     360      10       7     478       1       0
0       0       0       0       0       0       0       1      21       0
0
%FLAG ATOM_NAME
%FORMAT(20a4)
C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  H61 H62 O6
HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  H61 H62
O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6  H61
H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5  C6
H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5  O5
C6  H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5  H5
O5  C6  H61 H62 O6  HO6 C1  H1  O1  C2  H2  O2  HO2 C3  H3  O3  HO3 C4  H4  C5
H5  O5  C6  H61 H62 O6  HO6 C1  H1  C2  H2  C3  H3  C4  C5  H4  C6

0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
--
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00  0.00000000E+00
0.00000000E+00  0.00000000E+00  0.00000000E+00  0

In [283]:
def check_amber_crash(topology, coordinates, input_file, output_file, restart_file, executable, path):
    """
    Quickly checks to see if an AMBER minimization can succeed.    
    Parameters
    ----------
    topology : PDBFixer topology (probably others work, as well)
        Simulation topology
    coordinates : PDBFixer coordinates (others will work)
        Coordinates of the atoms in the topology
    output_file : str
        Output of the minimization
    restart_file : str
        Simulation restart final
    executable : str
        AMBER executable (sander is safe), must be in path
    path : str
        Directory of files
    """

    minimization = \
'''
&cntrl
    imin   = 1,
    maxcyc = 500,
    ncyc   = 250,
    ntb    = 1,
    igb    = 0,
    cut    = 12
/     
'''
    with open(path + input_file, 'w') as file:
        file.write(minimization)
    print("{4} -O -p {0} -c {1} -ref {1} -i {2} -o {3} -r {5}".format(topology, 
                            coordinates,
                            input_file,
                            output_file,
                            executable,
                            restart_file))

    sp.call("{4} -O -p {0} -c {1} -ref {1} -i {2} -o {3} -r {5}".format(topology, 
                            coordinates,
                            input_file,
                            output_file,
                            executable,
                            restart_file), cwd=path, shell=True)

In [284]:
def grep_output(output_file, path):
    p = sp.Popen(['cat', output_file], cwd=path, stdout=sp.PIPE)
    for line in p.stdout:
        print(line.decode("utf-8").strip(),)

In [285]:
check_amber_crash('solvated_smirnoff.prmtop', 'solvated_smirnoff.inpcrd', 'solvated_smirnoff.in', 
               'solvated_smirnoff.out', 'solvated_smirnoff.rst', 'sander', 'generated/')

sander -O -p solvated_smirnoff.prmtop -c solvated_smirnoff.inpcrd -ref solvated_smirnoff.inpcrd -i solvated_smirnoff.in -o solvated_smirnoff.out -r solvated_smirnoff.rst


In [286]:
grep_output('solvated_smirnoff.out', path='generated/')


-------------------------------------------------------
Amber 16 SANDER                              2016
-------------------------------------------------------

| Run on 02/05/2018 at 16:56:08

|   Executable path: sander
| Working directory: /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest/generated
|          Hostname: warthog

[-O]verwriting output

File Assignments:
|  MDIN: solvated_smirnoff.in
| MDOUT: solvated_smirnoff.out
|INPCRD: solvated_smirnoff.inpcrd
|  PARM: solvated_smirnoff.prmtop
|RESTRT: solvated_smirnoff.rst
|  REFC: solvated_smirnoff.inpcrd
| MDVEL: mdvel
| MDFRC: mdfrc
|  MDEN: mden
| MDCRD: mdcrd
|MDINFO: mdinfo
|  MTMD: mtmd
|INPDIP: inpdip
|RSTDIP: rstdip
|INPTRA: inptraj


Here is the input file:


&cntrl
imin   = 1,
maxcyc = 500,
ncyc   = 250,
ntb    = 1,
igb    = 0,
cut    = 12
/

--------------------------------------------------------------------------------
1.  RESOURCE   USE:
------------------------------------------------------------------